In [ ]:
% add the relevant functions from the src folder
addpath(genpath(fullfile(pwd, '..', 'src')))

## Setting the parameters about our experiment

We need to define the configuration structure where we put all our information.

In [ ]:
% define the folder where the data will be saved
cfg.dir.output = fullfile(pwd, 'output');

% define the name of the task
cfg.task.name = 'test task';

% can use the userInputs function to collect subject info
% cfg = userInputs;

% Or we can add those information manually directly
cfg.subject.subjectNb = 1;
cfg.subject.runNb = 1;

% if set to 'mri' then the data will be saved in the `func` folder
cfg.testingDevice = 'mri';

% to keep things quiet
cfg.verbose = false;


If set to `eeg` then the data will be saved in the `eeg` folder

```
cfg.testingDevice = 'eeg';
```

By default we assume you are running things on a behavioral PC with no eyetracker.

```
cfg.eyeTracker = false;
cfg.testingDevice = 'pc';
```

If the testing device is set to `pc` then the data will be saved in the `beh` folder

In [ ]:
% We can then initialize all the other fields. The ones we have already filled in will not be overwritten
cfg = checkCFG(cfg)

## Creating BIDS paths and filenames

In [ ]:
% create the filenames: this include a step to check that all the information is there: createFilename calls 
% checkCFG before running
[cfg] = createFilename(cfg);

### Output directory for this subject

Everything is put in a `source` folder to distinguish it from the `raw` that is usually reserved for the final 'true' BIDS data.

You can see that even though it was not specified a session folder and label is added to the folder structure and the filename.

In [ ]:
cfg.dir.outputSubject

### filename for events.tsv files

You will note that we are converting the name of the task to a `camelCase`.

`test task` --> `testTask`

In [ ]:
cfg.fileName.events

The fullpath for this file would be:

In [ ]:
fullfile(...
    cfg.dir.outputSubject, ...
    cfg.fileName.modality, ...
    cfg.fileName.events)

## Saving data to tsv file

### Setting things up

In [ ]:
clear cfg

cfg.dir.output = fullfile(pwd, 'output');

cfg.task.name = 'test task';

cfg.subject.subjectNb = 1;
cfg.subject.runNb = 1;

cfg.testingDevice = 'mri';

cfg.verbose = false;

% Create the filenames
[cfg] = createFilename(cfg);

% We can define what extra columns we want in our tsv file beyond the 
% BIDS holy trinity ('onset', 'duration', 'trial_type')

% Say we want to keep track of the type of target that what presented during a trial and of its position
logFile.extraColumns = {'target_position', 'target_type'};


### Preparing your data dictionnary

We can then initialize our logfile.

In [ ]:
logFile = saveEventsFile('init', cfg, logFile);

This will initialize a structure for each extra column.

In [ ]:
logFile.extraColumns.target_position.bids

You can then add information about each extraccolumns so they are saved in the JSON data dictionary that will go with the `.tsv` file.

In [ ]:
logFile.extraColumns.target_position.bids.Description = 'Position of the target on the screen';
logFile.extraColumns.target_position.bids.Levels = {'left', 'center', 'right'};

logFile.extraColumns.target_position.bids

### Adding data during the experiment

A couple of things to do before you start the main loop of your experiment.

In [ ]:
% create the output directory
mkdir(fullfile(cfg.dir.outputSubject, cfg.fileName.modality));
% open the file
logFile = saveEventsFile('open', cfg, logFile);

% This prints the header of the tsv file.


Calling `tree` from the linux command line to see that we got what we want

In [ ]:
!tree

2 things to note.

- A `_date-YYYYMMDDHHMM` suffix is appended to the file name. This is NOT accepted by the BIDS filenaming convention. This was introduced to prevent overwriting files by mistake in case some things were re-run. When you convert the source data into raw data, this date suffix should be removed (the `removeDataSuffix` is there for that).

- A json file that stores your data dictionnary was created.

Let's inpsect its content.

In [ ]:
!cat output/source/sub-001/ses-001/func/sub-001_ses-001_task-testTask_run-001_events_date-*.json

In [ ]:
% The information about 2 events that we want to save
% NOTE : If the user DOES NOT provide `onset`, `trial_type`, this events will be skipped.

% Trial one
logFile(1,1).onset = 2;
logFile(1,1).trial_type = 'motion_up';
logFile(1,1).duration = 1;
logFile(1,1).target_position = 'left';
logFile(1,1).target_type = 'audio';

% Trial two
logFile(2,1).onset = 6;
logFile(2,1).trial_type = 'static';
logFile(2,1).duration = 1.2;
logFile(2,1).target_position = 'right';
logFile(2,1).target_type = 'visual';

% add those 2 events to the events.tsv file

logFile = saveEventsFile('save', cfg, logFile);


The content of the `logfile` structure is emptied after every save so saving again will only throw a warning.

In [ ]:
logFile = saveEventsFile('save', cfg, logFile);

Any missing information will be replaced by `n/a`.

And will throw a warning. They can be quite verbose on octave.

In [ ]:
% New trial
logFile(1,1).onset = 10;
logFile(1,1).trial_type = 'motion_down';
logFile(1,1).duration = 1;
logFile(1,1).target_position = [];
logFile(1,1).target_type = 'audio';

logFile = saveEventsFile('save', cfg, logFile);

You can then close the file.

In [ ]:
% close the file
cfg.verbose = true; % set verbose in true if you want to know where things are
saveEventsFile('close', cfg, logFile);

### Inspecting the output

The `bids-matlab` toolbox comes as part of CPP_BIDS as a submodule so you can easily load the content of this tsv file.

In [ ]:
output = bids.util.tsvread( ...
    fullfile(...
    cfg.dir.outputSubject, ...
    cfg.fileName.modality, ...
    cfg.fileName.events))